In [1]:
from pandas_datareader import data
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import talib as ta1
import pandas_ta as ta
import datetime
import requests

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')


# Retrieve Trading data from houbi , 4hours

In [12]:
ticker = 'ethusdt'
base = 'https://api.huobi.pro'
url = '/market/history/kline?period=4hour&size=2000&symbol='+ ticker
final_url = base + url

reqs = requests.get(final_url)

df = pd.DataFrame(reqs.json()['data'])
df["Date"] = pd.to_datetime(df["id"], unit='s', origin='unix')
df["Date"] = df["Date"].dt.strftime("%m/%d/%Y %H:%M:%S")
df.drop('id',axis = 1, inplace = True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace = True)
df.rename(columns={'high':'High','low':'Low','open':'Open','close':'Close','vol':'Volume'},inplace=True)
df['Adj Close']= df['Close']
#df = df[['High','Low','Open','Close','Volume','Adj Close']]
df.sort_index(inplace = True)



# Generate Super Tranding 

In [3]:
df["sup"] = ta.supertrend(high = df["High"], low = df["Low"], close = df["Close"], length = 10, multiplier= 3).iloc[:,0]
df["supS"] = ta.supertrend(high = df["High"], low = df["Low"], close = df["Close"], length =34 , multiplier= 3).iloc[:,0]

## Super trend Buy Sell Signal

In [4]:
df["Buy Signal ST"] = 0
df["Sell Signal ST"] = 0 
n = 9 
for i in range (n,len(df)):
    if df["Close"][i-1] <= df["sup"][i-1] and df["Close"][i] > df["sup"][i]:
        df["Buy Signal ST"][i] = 1 
    if df["Close"][i-1] >= df["sup"][i-1] and df["Close"][i] < df["sup"][i]:
        df["Sell Signal ST"][i] = 1
 
df['Budget'] = 100.00

# Generate MACD

In [5]:
df['MACD_h'] = ta.macd(df['Close'],fast =12 , slow =26 ,signal =9 )['MACDh_12_26_9'] ## MACD insicator
df['SMA_200']=ta.sma(df['Close'],length=200) ## Simple Moving aAverage

In [6]:
## Calculate gradian 
df['M'] = np.nan
n = 2

for y in range(n,len(df)):
    df['M'][y] = (df['SMA_200'][y]- df['SMA_200'][y-n]) / n


## MACD buy sell signal

In [7]:
df['MACD Long'] = 0 

for j in range(len(df)):
    if df['MACD_h'][j] >0 :
        df['MACD Long'][j] = 1


# Back testing For super trend

In [8]:

'''
Buy_qty = 1
buying = False
cost = 0 

for x in range(1,len(df)):
    if df['Buy Signal ST'][x] == 1 and buying == False :
        cost = df['Budget'][x-1] * Buy_qty
        df['Budget'][x] = df['Budget'][x-1] * (1- Buy_qty)
        buy_price = df['Close'][x]
        buying = True

    else :
        df['Budget'][x] = df['Budget'][x-1]
    
    if df['Sell Signal ST'][x] == 1 and buying == True:
        temp_budget = df['Budget'][x-1]+ (cost * (df['Close'][x]- buy_price)/buy_price) +cost
        df['Budget'][x]=round(temp_budget,2)
        #print()
        buying = False

'''



"\nBuy_qty = 1\nbuying = False\ncost = 0 \n\nfor x in range(1,len(df)):\n    if df['Buy Signal ST'][x] == 1 and buying == False :\n        cost = df['Budget'][x-1] * Buy_qty\n        df['Budget'][x] = df['Budget'][x-1] * (1- Buy_qty)\n        buy_price = df['Close'][x]\n        buying = True\n\n    else :\n        df['Budget'][x] = df['Budget'][x-1]\n    \n    if df['Sell Signal ST'][x] == 1 and buying == True:\n        temp_budget = df['Budget'][x-1]+ (cost * (df['Close'][x]- buy_price)/buy_price) +cost\n        df['Budget'][x]=round(temp_budget,2)\n        #print()\n        buying = False\n\n"

# Back Test for ST+MACD

In [9]:
Buy_qty = 1 # use in percentage (times 100%)
buying = False
cost = 0 
Buying_ST = False
TP_ratio, SL_ratio =  0.015 , 0.01 # >>>> Ratio  1.5 : 1 , reference to TRI

for k in range(1, len(df)):
    ## to store Buy ST buysignal 
    if df['Buy Signal ST'][k] == 1:
        Buying_ST = True
    elif df['Sell Signal ST'][k] ==1 :
        Buying_ST = False 

    ## Triger buy if MACD_h >0 & super trend having buy signal
    if df['MACD Long'][k] ==1 and buying == False :#and df['M'][k]>2:
        if Buying_ST == True:
            cost = df['Budget'][k-1] * Buy_qty
            df['Budget'][k] = df['Budget'][k-1] * (1- Buy_qty)
            buy_price = df['Close'][k]
            buying = True
            Buying_ST = False

            ## create sell price
            #S_Price_TP = buy_price * (1 + TP_ratio)
            #S_Price_SL = buy_price * (1 - SL_ratio)
        else:
            df['Budget'][k] = df['Budget'][k-1]
    else :
        df['Budget'][k] = df['Budget'][k-1]


    ## Triger sell if super trend having sell signal  >>> to change buy and sell ratio 1:1.5
    if df['Sell Signal ST'][k] == 1 and buying == True:
        temp_budget = df['Budget'][k-1]+ (cost * (df['Close'][k]- buy_price)/buy_price) +cost
        df['Budget'][k]=round(temp_budget,2)
        buying = False



        '''    if buying == True: 
        if df['Close'][k]>= S_Price_TP or df['Close'][k]<= S_Price_SL:         # >>> to change buy and sell ratio 1:1.5
            temp_budget = df['Budget'][k-1]+ (cost * (df['Close'][k]- buy_price)/buy_price) +cost
            df['Budget'][k]=round(temp_budget,2)
            buying = False
        elif df['Sell Signal ST'][k] == 1:
            temp_budget = df['Budget'][k-1]+ (cost * (df['Close'][k]- buy_price)/buy_price) +cost
            df['Budget'][k]=round(temp_budget,2)
            buying = False'''




In [10]:
df_verify =df[['Close','Buy Signal ST','Sell Signal ST','SMA_200','Budget','M']]

In [11]:
##analysis purpose , Conclude if close below then dema169 high chance loss trade



- Long trade >> above dema169 , positive MACD histogram , super trend trigger buy signal  